Okay, it's time for us to start working with real data!  This project will require you to product **two** notebooks: one that processes data and dumps data to a pickle file, a second notebook that loads the pickle file and applies regression models to predict song popularity. Proposed due date: Friday October 26th, 5pm.

1. The following link is a fairly famous unprocessed data set which collected listening habits of approximately 1000 users:
http://www.dtic.upf.edu/~ocelma/MusicRecommendationDataset/lastfm-1K.html

    The data set is large (about 650 MB compressed), data is formatted one entry per line, tab-separated.  The first task is to process this data to recover some song statistics.  In particular, we would like to create (and save, using pickle), a pandas dataframe that contains:
    * the number of times a song has been played
    * percentage of the plays that came from users who self-identified as male
    * average age of the listener
    * percentage of plays that came from a user who played the song exactly once
    * percentage of plays that came from a user who played the song at least five times  
    
    If there any missing values, decide how you will deal with them, and summarize your approach.  

2. In this same pre-processing notebook, explore the data by generating some descriptive statistics.  After each code block, you should have summary statements, or code blocks should print out summary statements that are useful to a reader.

3. Complete this notebook by exporting the generated dataframe using pickle for posterity, but **don't commit your pickled file to the repository**.  Sometimes, it is more convenient to process the data locally rather than transmitting large processed data files.

4. In a new notebook, load the dataframe object.  Using k-fold cross-validation, apply a linear regression model.  Here, the response variable is the number of times a song has been played based on what we know about the listeners.  Be sure to return the model score on both the training and validation data.
5. Explain your results.  Specifically address, if the assumption of linear response to the input features was reasonable.  You will want to study a plot of the model's residual.  Ideally, the residuals are small in magnitude, and are evenly distributed about the true values.  What are some possible improvements that can be made?
6. Suppose we are instead curious about whether a song will be played by a user based on a demographics.  Apply a logistic regression model and report your results.

# Notebook 1

Before we begin the project, it would be better for us to import the package that we are going to use in the beginning to save time running the package when processing the code later on.
There are several package that are going to be used in this notebook.
1. import pandas as pd : adding pandas package to the project in order to support doing the analysis. It enables to provide a fast, flexible and expressive data structures with a good designed for analysis.
2. import numpy as np : is used to enabled different operation for example working with array.
3. import tarfile : to enable working with tar file.
4. import io : is a system based package to read the file.
5. import pickle : to enable operating pickle for the dataset.
6. import matplotlib.pyplot as plt : for plotting the data visually. Like providing graph.


In [1]:
import pandas as pd
import numpy as np
import tarfile
import io
import pickle
import matplotlib.pyplot as plt

As we will used data in the form of .tar file here, we have to enable the tarfile package in this project. Then we can do our command code to open the file, otherwise the file will remain close. The following code will enable the file to be accessed in this notebook.

In [2]:
tar = tarfile.open("data/lastfm-dataset-1K.tar.gz")
print(tar)

Now, the file is already enabled, we can check the details information of the file. As the tar file work like a zip file, we have to check what kind of files this tar file inside. We can check it by using the following code.

In [3]:
#tar = tarfile.open("sample.tar.gz", "r:gz")
for tarinfo in tar:
    print tarinfo.name, "is", tarinfo.size, "bytes in size and is",
    if tarinfo.isreg():
        print "a regular file."
    elif tarinfo.isdir():
        print "a directory."
    else:
        print "something else."
tar.close()

lastfm-dataset-1K is 0 bytes in size and is a directory.
lastfm-dataset-1K/userid-profile.tsv is 37842 bytes in size and is a regular file.
lastfm-dataset-1K/README.txt is 2235 bytes in size and is a regular file.
lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv is 2529193595 bytes in size and is a regular file.


We can see on the above output that there are several files that we can access through this tarfile. Since our first task is to process this data to recover some song statistics, then we can take the file that contains the information about user and the song. In this case, we can access the `userid-profile.tsv` and `userid-timestamp-artid-artname-traid-traname.tsv`. The following code will access the file and add it into dataframe for further uses.

In [5]:
#Do note that using error_bad_lines=False will cause the offending lines to be skipped
dfuserid = pd.read_csv("lastfm-dataset-1K/userid-profile.tsv", sep="\t", error_bad_lines=False)

colnames = ["User ID", "Timestamp", "Artist ID", "Artist Name", "Track ID", "Track Name"]
dftrack = pd.read_csv("lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv", sep="\t", 
                 error_bad_lines=False, header=None, names=colnames)

We can check the number of obervations we have and print the first 5 rows of the dataframe by using the following code.

In [6]:
print(dfuserid.shape)
dfuserid.head()
print(dftrack.shape)
dftrack.head()

(992, 5)
(19098862, 6)


,User ID,Timestamp,Artist ID,Artist Name,Track ID,Track Name
0,user_000001,2009-05-04T23:08:57Z,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish,NaN,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
1,user_000001,2009-05-04T13:54:10Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Composition 0919 (Live_2009_4_15)
2,user_000001,2009-05-04T13:52:04Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc2 (Live_2009_4_15)
3,user_000001,2009-05-04T13:42:52Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Hibari (Live_2009_4_15)
4,user_000001,2009-05-04T13:42:11Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc1 (Live_2009_4_15)


Since the data are in two separate dataframe, we can combine it into one dataframe because we would like to get the information of the user who accessed the song as well. We can also check whether each variable contains any null values in it or not. The following code will join the two dataframe and check the existence of null values in each variable. 

In [7]:
#join the two table into one
userjoin = dftrack.join(dfuserid.set_index('#id'), on=dftrack['User ID'])
print userjoin.shape #print the number of observations
userjoin.isna().any()

(19098862, 10)


User ID        False
Timestamp      False
Artist ID       True
Artist Name    False
Track ID        True
Track Name      True
gender          True
age             True
country         True
registered      True
dtype: bool

As we can see on the above output, the variables that shown `true` in the output means that particular variable have missing values in their field, and no missing values otherwise.
There are some missing data in the dataframe that we created based on user and song data. Before doing further operation, it would be best to deal with this missing data in the beginning. We can remove the null values from the dataset as it does not contains any information at all and we do not know to which categories it belong to. As such, we will deal with this null values in order to prevent it to impact our statistic. The following code will first check the missing data in the filed of `Track Name`, `age`, and `gender` as these variables will be our focus to develop our statistics.

## Deal with the missing values

We can delete the null values by using the following code.

In [8]:
dfdropnull = userjoin.dropna(axis=0, subset=['Track Name', 'age', 'gender'])

The above code, wil delete the null values based on the three field that was defined. Now we can check again whether the three field that we want to focus on still have any null values.

In [9]:
dfdropnull.isna().any()

User ID        False
Timestamp      False
Artist ID       True
Artist Name    False
Track ID        True
Track Name     False
gender         False
age            False
country         True
registered     False
dtype: bool

Now, the variables track name, age and gender does not have any missing values, means that it is ready for further command. We can also print again the first 5 rows and the observations in this new dataset after we drop the null values by using the following code.

In [12]:
dfdropnull.head()

,User ID,Timestamp,Artist ID,Artist Name,Track ID,Track Name,gender,age,country,registered
74123,user_000003,2009-05-04T10:29:04Z,ed442b01-8de1-4f0b-9188-e68b2960f050,Samuli Kemppi,NaN,Samuli Kemppi - January 2009,m,22.0,United States,"Oct 30, 2005"
74124,user_000003,2009-04-21T16:56:35Z,4730a989-8cf4-437b-9825-135f01644eab,Dirt Nasty,f8ef2c6a-e71f-4f57-a8d5-876835192b0a,True Hollywood Story,m,22.0,United States,"Oct 30, 2005"
74125,user_000003,2009-04-21T16:47:11Z,4730a989-8cf4-437b-9825-135f01644eab,Dirt Nasty,2c04113a-746b-491f-b999-892600730604,Gotta Leave This Town,m,22.0,United States,"Oct 30, 2005"
74126,user_000003,2009-04-21T16:32:54Z,5879a47f-c54f-4ebe-8d63-cdc9c6310dd2,Mickey Avalon,NaN,F***In Em All,m,22.0,United States,"Oct 30, 2005"
74127,user_000003,2009-04-21T16:27:08Z,056e4f3e-d505-4dad-8ec1-d04f521cbb56,Daft Punk,10cdacde-e54a-412a-863b-81d9ea6b3df2,Daftendirekt,m,22.0,United States,"Oct 30, 2005"


In [11]:
dfdropnull.shape

(5296879, 10)

# New Trial

In particular, we would like to create (and save, using pickle), a pandas dataframe that contains:
    * the number of times a song has been played
    * percentage of the plays that came from users who self-identified as male
    * average age of the listener
    * percentage of plays that came from a user who played the song exactly once
    * percentage of plays that came from a user who played the song at least five times  
    
    If there any missing values, decide how you will deal with them, and summarize your approach.  

2. In this same pre-processing notebook, explore the data by generating some descriptive statistics.  After each code block, you should have summary statements, or code blocks should print out summary statements that are useful to a reader.

3. Complete this notebook by exporting the generated dataframe using pickle for posterity, but **don't commit your pickled file to the repository**.  Sometimes, it is more convenient to process the data locally rather than transmitting large processed data files.

## Data Processing

Now the data is ready for further processing, we can take the dataframe that we already declared on the previous code to report the statistics of the song. Our focus is to create a dataframe that contains the following information:

   * the number of times a song has been played
   * percentage of the plays that came from users who self-identified as male
   * average age of the listener
   * percentage of plays that came from a user who played the song exactly once
   * percentage of plays that came from a user who played the song at least five times

As there are several operation need to be done to get the above mentioned statistics, we will separate the command in each of the requirement. First, we can get the total number of times a song has been played.

### 1. The number of times a song has been played

In [ ]:
We count the data based on the track name as we want to count how many times a song has been played.

In [13]:
#group and count the data based on track name, count the number of times a song has been played
numplayed = dfdropnull.groupby(['Track Name']).size().reset_index(name='Number of Played')
numplayed.sort_values(['Track Name'], ascending=[True])

,Track Name,Number of Played
0,\nuser_000125\t2007-11-23T04:25:49Z\t03ad1736-...,1
1,~Kuuhaku~,50
2,!,2
3,! (Foreword),38
4,! (Forward),6
5,! (Performed By Ghostface Killah And Rhythm Ro...,8
6,! (The Song Formerly Known As),1
7,! +,1
8,! / Sn10.1.Ai 0711,1
9,! / Sn2.Ai 0853,1


The above output shows the number of times a song has been played. For example, we can see that a song with the title ~Kuuhaku~ has been listened for 50 times by the user.

### 2. Percentage of the plays that came from users who self-identified as male

In this part, we want to get the male percentage that plays the song, so we can filter the data based on gender and take only the number of male that access the song. The following code will group the data based on track name and gender.

In [15]:
#group the data based on track name and gender
byGender = dfdropnull.groupby(['Track Name', 'gender']).size().reset_index(name='By Gender')
#check the number of observations
byGender.shape

(553387, 3)

The following code will add the `byGender` dataframe into the previous number of song dataframe in order to get access into the total number of a song has been played to get the ratio of male percentages.

In [16]:
numbjoin = byGender.join(numplayed.set_index(['Track Name']), on=byGender['Track Name'])
numbjoin

,Track Name,gender,By Gender,Number of Played
0,\nuser_000125\t2007-11-23T04:25:49Z\t03ad1736-...,f,1,1
1,~Kuuhaku~,f,50,50
2,!,m,2,2
3,! (Foreword),f,25,38
4,! (Foreword),m,13,38
5,! (Forward),m,6,6
6,! (Performed By Ghostface Killah And Rhythm Ro...,m,8,8
7,! (The Song Formerly Known As),m,1,1
8,! +,m,1,1
9,! / Sn10.1.Ai 0711,m,1,1


Next, we can select the male number from the dataset and put it into other dataframe.

In [19]:
maleuser = numbjoin[numbjoin['gender']=='m' ]
maleuser

,Track Name,gender,By Gender,Number of Played
2,!,m,2,2
4,! (Foreword),m,13,38
5,! (Forward),m,6,6
6,! (Performed By Ghostface Killah And Rhythm Ro...,m,8,8
7,! (The Song Formerly Known As),m,1,1
8,! +,m,1,1
9,! / Sn10.1.Ai 0711,m,1,1
10,! / Sn2.Ai 0853,m,1,1
11,! Que Vida! (Stereo Mix),m,2,2
13,!!!Yeah!!!(Reggeaton Remix),m,1,1


Now we have the total number of times a song has been played and the total numbe of times the song was played by male user. We can compute the percentage of male listener to the total number of song has been played.

In [20]:
maleuser['Male Percentage'] = (maleuser['By Gender'] / maleuser['Number of Played']) * 100
maleuser = maleuser.infer_objects() #convert datatypes
print maleuser.dtypes

maleuser['Male Percentage'] = maleuser['Male Percentage'].round(2)
maleuser

Track Name           object
gender               object
By Gender             int64
Number of Played      int64
Male Percentage     float64
dtype: object


C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Track Name,gender,By Gender,Number of Played,Male Percentage
2,!,m,2,2,100.00
4,! (Foreword),m,13,38,34.21
5,! (Forward),m,6,6,100.00
6,! (Performed By Ghostface Killah And Rhythm Ro...,m,8,8,100.00
7,! (The Song Formerly Known As),m,1,1,100.00
8,! +,m,1,1,100.00
9,! / Sn10.1.Ai 0711,m,1,1,100.00
10,! / Sn2.Ai 0853,m,1,1,100.00
11,! Que Vida! (Stereo Mix),m,2,2,100.00
13,!!!Yeah!!!(Reggeaton Remix),m,1,1,100.00


Since, we got the male percentage, we can then merger it into our final dataset which contains the number of times a song has been played, the numbe of listener who is identified as male and its percentage.

In [21]:
#merging the result together
summarydf = numplayed[['Track Name', 'Number of Played']].merge(maleuser[['Track Name', 'By Gender', 'Male Percentage']], on='Track Name', how='left')
summarydf

,Track Name,Number of Played,By Gender,Male Percentage
0,\nuser_000125\t2007-11-23T04:25:49Z\t03ad1736-...,1,NaN,NaN
1,~Kuuhaku~,50,NaN,NaN
2,!,2,2.0,100.00
3,! (Foreword),38,13.0,34.21
4,! (Forward),6,6.0,100.00
5,! (Performed By Ghostface Killah And Rhythm Ro...,8,8.0,100.00
6,! (The Song Formerly Known As),1,1.0,100.00
7,! +,1,1.0,100.00
8,! / Sn10.1.Ai 0711,1,1.0,100.00
9,! / Sn2.Ai 0853,1,1.0,100.00


We can see the male percentage for each song, and the value that is missing like `NaN` means that it was listened by female user. As we did not count the female ratio so we left it as null.

## Average age of the listener

In this part, we are going to compute the average age of the user based on each song with the following code.

In [23]:
meanage = dfdropnull.groupby(['Track Name'])['age'].mean().reset_index(name='AVG Age')
meanage.sort_values(['Track Name'], ascending=[True])


,Track Name,AVG Age
0,\nuser_000125\t2007-11-23T04:25:49Z\t03ad1736-...,20.000000
1,~Kuuhaku~,20.000000
2,!,30.000000
3,! (Foreword),24.526316
4,! (Forward),25.000000
5,! (Performed By Ghostface Killah And Rhythm Ro...,23.250000
6,! (The Song Formerly Known As),28.000000
7,! +,28.000000
8,! / Sn10.1.Ai 0711,29.000000
9,! / Sn2.Ai 0853,29.000000


Then, we can combined the result with our final data set.

In [24]:
summarydf = summarydf[['Track Name', 'Number of Played', 'By Gender', 'Male Percentage']].merge(meanage[['Track Name', 'AVG Age']], on='Track Name', how='left')
summarydf

,Track Name,Number of Played,By Gender,Male Percentage,AVG Age
0,\nuser_000125\t2007-11-23T04:25:49Z\t03ad1736-...,1,NaN,NaN,20.000000
1,~Kuuhaku~,50,NaN,NaN,20.000000
2,!,2,2.0,100.00,30.000000
3,! (Foreword),38,13.0,34.21,24.526316
4,! (Forward),6,6.0,100.00,25.000000
5,! (Performed By Ghostface Killah And Rhythm Ro...,8,8.0,100.00,23.250000
6,! (The Song Formerly Known As),1,1.0,100.00,28.000000
7,! +,1,1.0,100.00,28.000000
8,! / Sn10.1.Ai 0711,1,1.0,100.00,29.000000
9,! / Sn2.Ai 0853,1,1.0,100.00,29.000000


In [ ]:
Now, our final dataset has an additional column of average age of its user.

## Percentage of plays that came from a user who played the song exactly once

In this section, we are going to compute the percentage of song came from the user who played the song exactly once. First we can group our data based on the track name and user id and count the number of played based on each user for each song.

In [26]:
userplayed = dfdropnull.groupby(['Track Name', 'User ID']).size().reset_index(name='By User')
userplayed

,Track Name,User ID,By User
0,\nuser_000125\t2007-11-23T04:25:49Z\t03ad1736-...,user_000125,1
1,~Kuuhaku~,user_000033,50
2,!,user_000134,2
3,! (Foreword),user_000026,8
4,! (Foreword),user_000083,2
5,! (Foreword),user_000182,6
6,! (Foreword),user_000206,17
7,! (Foreword),user_000344,4
8,! (Foreword),user_000356,1
9,! (Forward),user_000306,6


In [ ]:
Then, we can select the user who played the song exactly once by using the following code.

In [27]:
played1 = userplayed[userplayed['By User']==1 ]
played1

,Track Name,User ID,By User
0,\nuser_000125\t2007-11-23T04:25:49Z\t03ad1736-...,user_000125,1
8,! (Foreword),user_000356,1
12,! (The Song Formerly Known As),user_000167,1
13,! +,user_000167,1
14,! / Sn10.1.Ai 0711,user_000053,1
15,! / Sn2.Ai 0853,user_000053,1
18,!!!Yeah!!!(Reggeaton Remix),user_000084,1
20,!@*$%#,user_000086,1
21,!@*$%#,user_000091,1
31,!Resistancia! (Bonus Track),user_000149,1


Next, we can compute how many user has played the song exactly once for each song.

In [28]:
sumplayed1 = played1.groupby(['Track Name']).size().reset_index(name='Played Once')
sumplayed1

,Track Name,Played Once
0,\nuser_000125\t2007-11-23T04:25:49Z\t03ad1736-...,1
1,! (Foreword),1
2,! (The Song Formerly Known As),1
3,! +,1
4,! / Sn10.1.Ai 0711,1
5,! / Sn2.Ai 0853,1
6,!!!Yeah!!!(Reggeaton Remix),1
7,!@*$%#,2
8,!Resistancia! (Bonus Track),1
9,"# 173 - The Stills, Novillero, Fred Eaglesmith",1


And then, merge the value into our final data set.

In [29]:
summarydf = summarydf.join(sumplayed1.set_index(['Track Name']), on=summarydf['Track Name'])
summarydf

,Track Name,Number of Played,By Gender,Male Percentage,AVG Age,Played Once
0,\nuser_000125\t2007-11-23T04:25:49Z\t03ad1736-...,1,NaN,NaN,20.000000,1.0
1,~Kuuhaku~,50,NaN,NaN,20.000000,NaN
2,!,2,2.0,100.00,30.000000,NaN
3,! (Foreword),38,13.0,34.21,24.526316,1.0
4,! (Forward),6,6.0,100.00,25.000000,NaN
5,! (Performed By Ghostface Killah And Rhythm Ro...,8,8.0,100.00,23.250000,NaN
6,! (The Song Formerly Known As),1,1.0,100.00,28.000000,1.0
7,! +,1,1.0,100.00,28.000000,1.0
8,! / Sn10.1.Ai 0711,1,1.0,100.00,29.000000,1.0
9,! / Sn2.Ai 0853,1,1.0,100.00,29.000000,1.0


Finally, as it is in the sam dataset, we can directly compute the percentage.

In [30]:
summarydf['Once Percentage'] = (summarydf['Played Once'] / summarydf['Number of Played']) * 100
summarydf = summarydf.infer_objects() #convert datatypes
print summarydf.dtypes

summarydf['Once Percentage'] = summarydf['Once Percentage'].round(2)
summarydf

Track Name           object
Number of Played      int64
By Gender           float64
Male Percentage     float64
AVG Age             float64
Played Once         float64
Once Percentage     float64
dtype: object


C:\ProgramData\Anaconda2\lib\site-packages\pandas\core\series.py:1826: RuntimeWarning: invalid value encountered in rint
  result = com._values_from_object(self).round(decimals)


,Track Name,Number of Played,By Gender,Male Percentage,AVG Age,Played Once,Once Percentage
0,\nuser_000125\t2007-11-23T04:25:49Z\t03ad1736-...,1,NaN,NaN,20.000000,1.0,100.00
1,~Kuuhaku~,50,NaN,NaN,20.000000,NaN,NaN
2,!,2,2.0,100.00,30.000000,NaN,NaN
3,! (Foreword),38,13.0,34.21,24.526316,1.0,2.63
4,! (Forward),6,6.0,100.00,25.000000,NaN,NaN
5,! (Performed By Ghostface Killah And Rhythm Ro...,8,8.0,100.00,23.250000,NaN,NaN
6,! (The Song Formerly Known As),1,1.0,100.00,28.000000,1.0,100.00
7,! +,1,1.0,100.00,28.000000,1.0,100.00
8,! / Sn10.1.Ai 0711,1,1.0,100.00,29.000000,1.0,100.00
9,! / Sn2.Ai 0853,1,1.0,100.00,29.000000,1.0,100.00


Now, we can see that our final data set has an additional of percentage came from the user who played the song exactly once. The `NaN` value in the data set means that there is no user who only played the song exactly once, all the user like the song and played it more than once.

## Percentage of plays that came from a user who played the song at least five times 

In this section, we are going to compute the percentage of song came from the user who played the song at least five times. First, we can used the dataset from the previous section and select the user who played the song at least 5 times.

In [31]:
played5 = userplayed[userplayed['By User'] >= 5 ]
played5

,Track Name,User ID,By User
1,~Kuuhaku~,user_000033,50
3,! (Foreword),user_000026,8
5,! (Foreword),user_000182,6
6,! (Foreword),user_000206,17
9,! (Forward),user_000306,6
11,! (Performed By Ghostface Killah And Rhythm Ro...,user_000351,5
22,!@*$%#,user_000251,14
24,!Fire In The Hole!,user_000349,9
28,!Kamikaze!,user_000349,7
34,#,user_000306,12


Next, we can compute how many user has played the song exactly once for each song.

In [32]:
sumplayed5 = played5.groupby(['Track Name']).size().reset_index(name='Played Five')
sumplayed5

,Track Name,Played Five
0,~Kuuhaku~,1
1,! (Foreword),3
2,! (Forward),1
3,! (Performed By Ghostface Killah And Rhythm Ro...,1
4,!@*$%#,1
5,!Fire In The Hole!,1
6,!Kamikaze!,1
7,#,1
8,#1,3
9,#1 Bad Boy,1


And then, merge the value into our final data set.

In [33]:
summarydf = summarydf.join(sumplayed5.set_index(['Track Name']), on=summarydf['Track Name'])
summarydf

,Track Name,Number of Played,By Gender,Male Percentage,AVG Age,Played Once,Once Percentage,Played Five
0,\nuser_000125\t2007-11-23T04:25:49Z\t03ad1736-...,1,NaN,NaN,20.000000,1.0,100.00,NaN
1,~Kuuhaku~,50,NaN,NaN,20.000000,NaN,NaN,1.0
2,!,2,2.0,100.00,30.000000,NaN,NaN,NaN
3,! (Foreword),38,13.0,34.21,24.526316,1.0,2.63,3.0
4,! (Forward),6,6.0,100.00,25.000000,NaN,NaN,1.0
5,! (Performed By Ghostface Killah And Rhythm Ro...,8,8.0,100.00,23.250000,NaN,NaN,1.0
6,! (The Song Formerly Known As),1,1.0,100.00,28.000000,1.0,100.00,NaN
7,! +,1,1.0,100.00,28.000000,1.0,100.00,NaN
8,! / Sn10.1.Ai 0711,1,1.0,100.00,29.000000,1.0,100.00,NaN
9,! / Sn2.Ai 0853,1,1.0,100.00,29.000000,1.0,100.00,NaN


In [34]:
summarydf['Fifth Percentage'] = (summarydf['Played Five'] / summarydf['Number of Played']) * 100
summarydf = summarydf.infer_objects() #convert datatypes
print summarydf.dtypes

summarydf['Fifth Percentage'] = summarydf['Fifth Percentage'].round(2)
summarydf['AVG Age'] = summarydf['AVG Age'].round(2)
summarydf

Track Name           object
Number of Played      int64
By Gender           float64
Male Percentage     float64
AVG Age             float64
Played Once         float64
Once Percentage     float64
Played Five         float64
Fifth Percentage    float64
dtype: object


,Track Name,Number of Played,By Gender,Male Percentage,AVG Age,Played Once,Once Percentage,Played Five,Fifth Percentage
0,\nuser_000125\t2007-11-23T04:25:49Z\t03ad1736-...,1,NaN,NaN,20.00,1.0,100.00,NaN,NaN
1,~Kuuhaku~,50,NaN,NaN,20.00,NaN,NaN,1.0,2.00
2,!,2,2.0,100.00,30.00,NaN,NaN,NaN,NaN
3,! (Foreword),38,13.0,34.21,24.53,1.0,2.63,3.0,7.89
4,! (Forward),6,6.0,100.00,25.00,NaN,NaN,1.0,16.67
5,! (Performed By Ghostface Killah And Rhythm Ro...,8,8.0,100.00,23.25,NaN,NaN,1.0,12.50
6,! (The Song Formerly Known As),1,1.0,100.00,28.00,1.0,100.00,NaN,NaN
7,! +,1,1.0,100.00,28.00,1.0,100.00,NaN,NaN
8,! / Sn10.1.Ai 0711,1,1.0,100.00,29.00,1.0,100.00,NaN,NaN
9,! / Sn2.Ai 0853,1,1.0,100.00,29.00,1.0,100.00,NaN,NaN


Now, we can see that our final data set has an additional of percentage came from the user who played the song at least 5 times. The `NaN` value in the data set means that there is no user who played the song more than 4 times.

# Dealing with a song who only have female listener

If there any missing values, decide how you will deal with them, and summarize your approach.  

2. In this same pre-processing notebook, explore the data by generating some descriptive statistics.  After each code block, you should have summary statements, or code blocks should print out summary statements that are useful to a reader.

3. Complete this notebook by exporting the generated dataframe using pickle for posterity, but **don't commit your pickled file to the repository**.  Sometimes, it is more convenient to process the data locally rather than transmitting large processed data files.

As we already said before, in some variables there will be a null values. For example in the percentage of male listener, the female listener will appear as null. In the percentage of song played exactly once, the listener who listened to the song more than once will apppear as null values. Also, in the percentage of song has been played at least 5 times, the listener who played the song less than 5 will appear null. Therefore, we can set the null value as `0` to mitigate having empty values. The following code will defined the null values as `0`.

In [35]:
# set missing values as 0
resetsummary = summarydf.fillna(0)
resetsummary

,Track Name,Number of Played,By Gender,Male Percentage,AVG Age,Played Once,Once Percentage,Played Five,Fifth Percentage
0,\nuser_000125\t2007-11-23T04:25:49Z\t03ad1736-...,1,0.0,0.00,20.00,1.0,100.00,0.0,0.00
1,~Kuuhaku~,50,0.0,0.00,20.00,0.0,0.00,1.0,2.00
2,!,2,2.0,100.00,30.00,0.0,0.00,0.0,0.00
3,! (Foreword),38,13.0,34.21,24.53,1.0,2.63,3.0,7.89
4,! (Forward),6,6.0,100.00,25.00,0.0,0.00,1.0,16.67
5,! (Performed By Ghostface Killah And Rhythm Ro...,8,8.0,100.00,23.25,0.0,0.00,1.0,12.50
6,! (The Song Formerly Known As),1,1.0,100.00,28.00,1.0,100.00,0.0,0.00
7,! +,1,1.0,100.00,28.00,1.0,100.00,0.0,0.00
8,! / Sn10.1.Ai 0711,1,1.0,100.00,29.00,1.0,100.00,0.0,0.00
9,! / Sn2.Ai 0853,1,1.0,100.00,29.00,1.0,100.00,0.0,0.00


# Descriptive Statistics

In addition to the above statistics, we can do some other statistics based on the original data. For example we can know the count and number of participants.

In [42]:
dfdropnull[['User ID', 'Track Name', 'Artist Name']].describe()

,User ID,Track Name,Artist Name
count,5296879,5296879,5296879
unique,271,454688,70472
top,user_000233,Intro,Radiohead
freq,117488,4585,34575


On the above output, we can know that the total observation for user id, track name, and the artist name is 5296879 for each. It also shows that there are 271 different user, 454688 different song from 70472 artists.

In [37]:
dfdropnull.describe(include='all')

,User ID,Timestamp,Artist ID,Artist Name,Track ID,Track Name,gender,age,country,registered
count,5296879,5296879,5138209,5296879,4694916,5296879,5296879,5.296879e+06,5208066,5296879
unique,271,5157105,51611,70472,433584,454688,2,NaN,44,230
top,user_000233,2009-04-07T21:52:40Z,a74b1b7f-71a5-4011-9441-d0b5e4122711,Radiohead,153d8ca0-dc23-4548-b64f-2c49db7f30db,Intro,m,NaN,United States,"Jul 20, 2005"
freq,117488,161,34575,34575,2367,4585,3466064,NaN,1253710,117488
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.495218e+01,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.100131e+00,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000e+00,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.100000e+01,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.400000e+01,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.800000e+01,NaN,NaN


In regards to our precessed data, we can know the details decriptive statistics by using the following code.

In [40]:
resetsummary.describe(include='all')

,Track Name,Number of Played,By Gender,Male Percentage,AVG Age,Played Once,Once Percentage,Played Five,Fifth Percentage
count,454688,454688.000000,454688.000000,454688.000000,454688.000000,454688.000000,454688.000000,454688.000000,454688.000000
unique,454688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,Power Outage At The Pool,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,11.649481,7.622950,67.217073,26.445792,1.136520,43.088301,0.588760,2.827662
std,NaN,38.802318,26.977751,42.568355,6.716780,2.026046,46.369161,1.810893,5.110601
min,NaN,1.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,1.000000,1.000000,14.290000,22.000000,0.000000,0.000000,0.000000,0.000000
50%,NaN,3.000000,1.000000,100.000000,25.000000,1.000000,15.150000,0.000000,0.000000
75%,NaN,8.000000,5.000000,100.000000,29.330000,1.000000,100.000000,1.000000,4.550000


On the above output, we can see the number of observations, the mean, median, min, max, and the quantiles of the data.

## Pickling the processed dataframe

In [39]:
pickle_out = open('data.pkl', 'wb')
pickle.dump(resetsummary, pickle_out, pickle.HIGHEST_PROTOCOL)
pickle_out.close()